In [ ]:
import os
import sys
import calendar
from datetime import datetime, timedelta
import pandas as pd
import ipywidgets as widgets
import yfinance as yf
import pandas as pd
from IPython.display import display
from pathlib import Path

from openbb_terminal import OpenBBFigure
from openbb_terminal.core.config.paths import MISCELLANEOUS_DIRECTORY

In [ ]:
df = pd.read_csv(MISCELLANEOUS_DIRECTORY / "futures" / "futures.csv")

# These are the symbols that futures use for each month
months = ["F", "G", "H", "J", "K", "M", "N", "Q", "U", "V", "X", "Z"]

In [ ]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


def format_plotly(fig: OpenBBFigure, data, chart, calc=None):
    fig.update_yaxes(title=None)
    fig.update_xaxes(title=None)
    height = 500 if chart == "main" else 300
    fig.update_layout(
        margin=dict(l=0, r=10, t=10, b=10),
        autosize=False,
        width=900,
        height=height,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )


def create_line(visual, x, y, name, fig: OpenBBFigure):
    if visual == "line":
        fig.add_scatter(x=x, y=y, mode="lines", name=name)  # , connectgaps=True
    if visual == "scatter":
        fig.add_scatter(x=x, y=y, mode="markers", name=name)
    if visual == "candle":
        fig.add_candlestick(
            x=x,
            open=y["Open"],
            close=y["Close"],
            high=y["High"],
            low=y["Low"],
            name=name,
        )


def show_fig(fig: OpenBBFigure):
    config = {"showTips": False, "scrollZoom": True, "displaylogo": False}
    if os.environ.get("SERVER_SOFTWARE", "jupyter").startswith("voila"):
        fig.show(config=config, renderer="notebook")
    else:
        fig.show(config=config)


def build_ticker(ticker: str, month: int, year: int) -> str:
    if ticker:
        row = df[df["Ticker"] == f"{ticker.split(':')[0]}"].iloc[0]
        row = row.to_dict()
        the_tick = row["Ticker"].replace("=F", "")
        return f"{the_tick}{months[month-1]}{str(year)[-2:]}.{row['Exchange']}"
    return ""


def next_ticker(ticker: str) -> str:
    symbol, exchange = ticker.split(".")
    month = symbol[-3]
    if month == "Z":
        new_month = "F"
        new_year = int(symbol[-2:]) + 1
    else:
        index = months.index(month)
        new_month = months[index + 1]
        new_year = int(symbol[-2:])
    return f"{symbol[:-3]}{new_month}{new_year}.{exchange}"


def get_column(df: pd.DataFrame, column: str):
    sub_df = df.xs(column, level=1, axis=1, drop_level=False)
    sub_df.columns = sub_df.columns.to_flat_index().map(lambda x: x[0])
    sub_df = sub_df.dropna(how="all")
    sub_df = sub_df.sort_index()
    if sub_df.empty:
        return None
    return sub_df


def get_date(x: str) -> datetime:
    ticker, _ = x.split(".")
    month_str = ticker[-3]
    month = months.index(month_str) + 1
    year = int(ticker[-2:]) + 2000
    day = calendar.monthrange(year, month)
    return datetime(year, month, day[1])

In [ ]:
class Chart:
    def __init__(self):
        self.tickers = {}
        self.last_ticker = ""

    def create_stock(self, chart_type, contracts, ticker):
        if not ticker:
            return

        if self.last_ticker != ticker:
            now = datetime.now()
            clean_ticker = build_ticker(ticker, now.month, now.year)
            if clean_ticker:
                raw_tickers = [clean_ticker]
                for _ in range(36):
                    new_ticker = next_ticker(raw_tickers[-1])
                    raw_tickers.append(new_ticker)
                    raw_ticker = ",".join(raw_tickers)
                with HiddenPrints():
                    # ,period="max"
                    dfs = yf.download(raw_ticker, progress=False)
                self.tickers = {
                    x: {"data": get_column(dfs, x), "date": get_date(x)}
                    for x in raw_tickers
                }
            self.last_ticker = ticker

        fig = OpenBBFigure()
        if chart_type[0][:4] == "Hist":
            for i, (key, item) in enumerate(self.tickers.items()):
                if i == contracts:
                    break
                result = item["data"]
                if result is not None:
                    create_line(
                        visual="line",
                        x=result.index,
                        y=result["Adj Close"],
                        name=key,
                        fig=fig,
                    )
        else:
            x = []
            y = []
            for _, value in self.tickers.items():
                if len(x) > contracts:
                    break
                if value["data"] is None:
                    continue
                x.append(value["date"])
                y.append(value["data"]["Adj Close"].iloc[-1])
            create_line(visual="line", x=x, y=y, name="Future Curve", fig=fig)

        format_plotly(fig, "Close", "main", "raw")
        show_fig(fig)

In [ ]:
category = df["Category"].unique().tolist()
exchange = df[df["Category"].isin([category[0]])]["Exchange"].unique().tolist()
filt_df = df[df["Category"].isin([category[0]]) & df["Exchange"].isin([exchange[0]])]
tickers_raw = filt_df[["Ticker", "Description"]].values.tolist()
tickers = [f"{x.replace('=F', '')}: {y}" for x, y in tickers_raw]

category.sort()
exchange.sort()
tickers.sort()

chart_type = ["Historical Time Series", "Future Curve"]
cat_widget = widgets.SelectMultiple(options=category, value=[category[0]])
exch_widget = widgets.SelectMultiple(options=exchange, value=[exchange[0]])
tickers_widget = widgets.Select(options=tickers, value=tickers[0])
chart_widget = widgets.SelectMultiple(options=chart_type, value=[chart_type[0]])


def on_change(change):
    cat_value = cat_widget.value
    exch_value = exch_widget.value

    # Filter exchange widget
    exch_filtered = df[df["Category"].isin(cat_value)]["Exchange"].unique().tolist()
    exch_filtered.sort()
    exch_widget.options = exch_filtered

    # Filter ticker widget
    filtered = df[df["Category"].isin(cat_value) & df["Exchange"].isin(exch_value)]
    tick_raw = filtered[["Ticker", "Description"]].values.tolist()
    tick = [f"{x.replace('=F', '')}: {y}" for x, y in tick_raw]
    tick.sort()
    tickers_widget.options = tick


cat_widget.observe(on_change)
exch_widget.observe(on_change)

contracts_widget = widgets.Dropdown(
    options=list(range(1, 25)), value=6, description="Contracts"
)

controls = widgets.HBox(
    [cat_widget, exch_widget, tickers_widget, chart_widget, contracts_widget],
    layout=widgets.Layout(width="90%"),
)
chart = Chart()
stocks_view = widgets.interactive_output(
    chart.create_stock,
    {
        "chart_type": chart_widget,
        "contracts": contracts_widget,
        "ticker": tickers_widget,
    },
)


title_html = "<h1>Futures Analysis Dashboard</h1>"

app_contents = [widgets.HTML(title_html), controls, stocks_view]
app = widgets.VBox(app_contents)
display(app)